In [16]:
import pandas as pd
import numpy as np

In [5]:
TESTSET_PATH = '../datasets/cleaned/test_set_for_evaluating_bert.csv'
test_df = pd.read_csv(TESTSET_PATH)

In [2]:
df = pd.read_csv('../datasets/bert_predictions/bert_predictions_weighted.csv')

In [7]:
def truncate(text, max_len=256):
    t = text[:max_len]
    t += '...'
    return t

In [8]:
df['text'] = [truncate(t) for t in test_df['text']]

In [10]:
df.drop(columns='Unnamed: 0', inplace=True)

,prediction,true,energy,text
0,18,0,"[-1.1461588, -4.1679974, -0.030044055, -5.0421...",can I replace Badger 1 1/3 with a Badger 5 1/2...
1,0,0,"[0.7836062, -4.93279, -3.2483032, -4.626629, -...",Can someone tell me if this Badger 5 disposer ...
2,18,0,"[-0.2254519, -4.5402284, -0.5819822, -3.545577...",My present disposal has a 3 bolt connection. W...
3,10,0,"[-1.6296536, -2.171615, -4.1076536, -4.0729985...",can you bake corn bread on this without it bur...
4,0,0,"[2.2435298, -3.9515293, -2.6414902, -3.575723,...","I need a device to let my dryer vent out, I li..."
...,...,...,...,...
127096,20,20,"[-7.1342626, -4.808668, -3.8864925, -5.2219462...",How do I get this to work for two players on p...
127097,7,20,"[-5.082524, -3.976024, -2.4956865, -4.4355555,...",how many gig on the hard drive It is a 500gb h...
127098,20,20,"[-4.3944526, -4.556605, -2.9484854, -3.1850255...",Does the kinect-less version still play blu ra...
127099,20,20,"[-7.7869315, -5.3577085, -4.101351, -4.7853866...",Are there offline options for the games for Xb...


In [11]:
df.loc[0,'text']

'can I replace Badger 1 1/3 with a Badger 5 1/2 - with same connections? Plumbing connections will vary with different models. Usually the larger higher amp draw wil not affect the wiring, the disposals are designed to a basic standard setup common to all b...'

In [21]:
def strings_to_floats(string):
    t= string[1:-1]
    t = np.fromstring(t, dtype=float, sep=', ')
    return t

In [22]:
strings_to_floats(df.loc[0,'energy'])

array([-1.1461588 , -4.1679974 , -0.03004406, -5.042155  , -5.5158677 ,
       -3.3954325 , -6.6709533 , -1.226475  , -8.255474  , -3.7266235 ,
       -2.6102517 ,  0.05398713, -2.76479   , -2.8700466 , -1.6670114 ,
       -3.310149  , -6.0924106 , -2.594835  ,  1.4618556 , -4.152668  ,
       -6.1064296 ])

In [28]:
def rank_preds(energy):
    return [x[1] for x in sorted(zip(strings_to_floats(energy), list(range(0,22))), reverse=True)]

In [31]:
df['rank'] = [rank_preds(x) for x in df['energy']]

In [33]:
df.drop(columns='energy', inplace=True)

In [35]:
df['true_pos'] = [rank.index(true) for true, rank in zip(df['true'], df['rank'])]

In [37]:
df['correct'] = df['true'] == df['prediction']

In [38]:
df

,prediction,true,text,rank,true_pos,correct
0,18,0,can I replace Badger 1 1/3 with a Badger 5 1/2...,"[18, 11, 2, 0, 7, 14, 17, 10, 12, 13, 15, 5, 9...",3,False
1,0,0,Can someone tell me if this Badger 5 disposer ...,"[0, 18, 13, 10, 11, 7, 9, 14, 2, 17, 4, 5, 15,...",0,True
2,18,0,My present disposal has a 3 bolt connection. W...,"[18, 0, 11, 2, 14, 10, 15, 7, 17, 9, 3, 19, 13...",1,False
3,10,0,can you bake corn bread on this without it bur...,"[10, 14, 8, 9, 0, 4, 17, 1, 11, 15, 18, 19, 3,...",4,False
4,0,0,"I need a device to let my dryer vent out, I li...","[0, 10, 18, 14, 9, 17, 11, 4, 2, 15, 3, 7, 1, ...",0,True
...,...,...,...,...,...,...
127096,20,20,How do I get this to work for two players on p...,"[20, 7, 19, 5, 13, 12, 17, 18, 16, 11, 10, 2, ...",0,True
127097,7,20,how many gig on the hard drive It is a 500gb h...,"[7, 12, 5, 16, 11, 20, 13, 19, 2, 18, 17, 9, 1...",5,False
127098,20,20,Does the kinect-less version still play blu ra...,"[20, 7, 5, 19, 12, 17, 16, 18, 13, 9, 14, 2, 1...",0,True
127099,20,20,Are there offline options for the games for Xb...,"[20, 7, 19, 5, 13, 17, 16, 12, 2, 10, 18, 11, ...",0,True


In [39]:
def get_samples(data, n=10):
    d = data.copy(deep=True)
    d = d.reset_index(drop=True)
    idx = np.round(np.linspace(0, d.shape[0] - 1, n)).astype(int)
    sample = d.iloc[idx]
    return sample

In [52]:
correct_samples = pd.DataFrame()
wrong_samples = pd.DataFrame()

for i in range(21):
    correct_samples = correct_samples.append(get_samples(df.groupby(['true', 'correct']).get_group((i, True))))
    wrong_samples = wrong_samples.append(get_samples(df.groupby(['true', 'correct']).get_group((i, False))))    

In [54]:
correct_samples.reset_index(drop=True, inplace=True)
wrong_samples.reset_index(drop=True, inplace=True)

In [55]:
correct_samples

,prediction,true,text,rank,true_pos,correct
0,0,0,Can someone tell me if this Badger 5 disposer ...,"[0, 18, 13, 10, 11, 7, 9, 14, 2, 17, 4, 5, 15,...",0,True
1,0,0,why did my humidifier stop emitting vapor? I c...,"[0, 10, 18, 9, 11, 4, 14, 17, 19, 2, 3, 15, 8,...",0,True
2,0,0,Is this compatible with GE Oven JK955SOF3SS? I...,"[0, 10, 18, 11, 14, 17, 2, 1, 9, 7, 19, 13, 8,...",0,True
3,0,0,The GE broil element I am looking for is part ...,"[0, 10, 18, 14, 2, 11, 17, 9, 7, 15, 3, 13, 5,...",0,True
4,0,0,Could failure of this part cause a slow leak a...,"[0, 10, 2, 18, 14, 11, 3, 17, 15, 7, 9, 5, 4, ...",0,True
...,...,...,...,...,...,...
205,20,20,is the controller region free ? can it be used...,"[20, 7, 19, 5, 12, 17, 13, 16, 9, 10, 18, 11, ...",0,True
206,20,20,"multiplayer online, Can you four people (frien...","[20, 7, 19, 12, 5, 16, 13, 17, 18, 10, 2, 11, ...",0,True
207,20,20,Anyone know if there's any issues using this w...,"[20, 7, 5, 12, 13, 11, 18, 16, 2, 10, 17, 19, ...",0,True
208,20,20,I am a real novice. What is a steam account? A...,"[20, 16, 19, 7, 13, 12, 17, 1, 5, 9, 11, 10, 3...",0,True


In [56]:
wrong_samples

,prediction,true,text,rank,true_pos,correct
0,18,0,can I replace Badger 1 1/3 with a Badger 5 1/2...,"[18, 11, 2, 0, 7, 14, 17, 10, 12, 13, 15, 5, 9...",3,False
1,2,0,Can the bulbs we swapped out at home for LED b...,"[2, 18, 15, 14, 7, 0, 17, 10, 11, 12, 9, 4, 5,...",5,False
2,13,0,"wheels Yes, there are two wheels on it making ...","[13, 17, 10, 3, 9, 14, 19, 2, 11, 18, 15, 1, 6...",14,False
3,14,0,Does the packaging that comes with this supply...,"[14, 18, 11, 17, 10, 0, 2, 15, 19, 7, 9, 13, 3...",5,False
4,10,0,Where is it made? Made in the good ole USA !...,"[10, 17, 4, 9, 7, 18, 5, 14, 12, 19, 15, 13, 3...",17,False
...,...,...,...,...,...,...
205,7,20,"Because this is listed as ""new,"" does it inclu...","[7, 20, 2, 5, 17, 10, 18, 11, 12, 13, 14, 9, 1...",1,False
206,5,20,Will the Wii pad still fit if it has a silicon...,"[5, 7, 20, 3, 2, 13, 17, 9, 19, 12, 10, 18, 15...",2,False
207,5,20,how many cords to i have to deal with? One sho...,"[5, 7, 18, 4, 10, 9, 12, 13, 14, 11, 17, 2, 0,...",16,False
208,9,20,can you get fellow buchers for loging Yes...,"[9, 13, 19, 14, 18, 10, 17, 11, 7, 2, 8, 1, 16...",19,False


In [58]:
correct_samples.to_json('../datasets/evaluated_samples/correct_samples.json', orient='records')

In [59]:
wrong_samples.to_json('../datasets/evaluated_samples/wrong_samples.json', orient='records')